In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPooling2D
import pickle
from keras.optimizers import Adam
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import time
import tensorflow as tf

Using TensorFlow backend.


In [2]:
start_time = time.clock()
np.random.seed(7)
random.seed(7)

In [3]:
tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
sess = print(tf.Session(config=tf.ConfigProto(log_device_placement=True)))

filename = '../../data/output/integrated_data_done.csv'

input_data = pd.read_csv(filename)

In [4]:
data = input_data[:50000]
data.shape

(50000, 381)

In [5]:
data.isnull().any()

route                                              False
tripNum                                            False
shapeSequence                                      False
shapeLat                                           False
shapeLon                                           False
distanceTraveledShape                              False
busCode                                            False
gpsPointId                                         False
gpsLat                                             False
gpsLon                                             False
distanceToShapePoint                               False
stopPointId                                        False
precipitation                                      False
alertRoadType                                      False
alertConfidence                                    False
alertNComments                                     False
alertNImages                                       False
alertNThumbsUp                 

In [22]:
del input_data

In [6]:
# converting input data format to multidimensional matrices

Total_InstanceNumber = len(data)

tripNum = data["tripNum"]
gpsPointId = data["gpsPointId"]
month = data["MONTH(gps_datetime)"]
distanceTraveledShape = data["distanceTraveledShape"]
alertConfidence = data["alertConfidence"]
bb = data["busBunching"]

Threshold = 1000
TotalInput = np.zeros((Total_InstanceNumber, 1, Threshold, 5), dtype=float)
FinalLabel = np.zeros((Total_InstanceNumber, 1), dtype=int)
counter = 0

for i in range(Total_InstanceNumber):
    TotalInput[i, 0, 0:i, 0] = tripNum[i]
    TotalInput[i, 0, 0:i, 1] = gpsPointId[i]
    TotalInput[i, 0, 0:i, 2] = month[i]
    TotalInput[i, 0, 0:i, 3] = distanceTraveledShape[i]
    TotalInput[i, 0, 0:i, 4] = alertConfidence[i]
    FinalLabel[i, 0] = bb[i]

TotalInput = TotalInput[:Total_InstanceNumber, :, :, :]
FinalLabel = FinalLabel[:Total_InstanceNumber, 0]

In [9]:
# input_file = pd.read_csv(filename, index_col=None, header=0)

# input_file.head()

,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,...,YEAR(alertDateTime),YEAR(jamUpdateDateTime),MONTH(gps_datetime),MONTH(precipitationTime),MONTH(alertDateTime),MONTH(jamUpdateDateTime),WEEKDAY(gps_datetime),WEEKDAY(precipitationTime),WEEKDAY(alertDateTime),WEEKDAY(jamUpdateDateTime)
0,68,1,Route_160,low_frequency,12573,-8.036592,-34.975683,785.0,12021,27.0,...,NaN,NaN,12,12,NaN,NaN,0,0,NaN,NaN
1,68,1,Route_160,low_frequency,12581,-8.034150,-34.960756,2794.0,12021,33.0,...,2018.0,NaN,12,12,12.0,NaN,0,0,0.0,NaN
2,68,1,Route_160,low_frequency,12582,-8.032878,-34.959775,2971.0,12021,NaN,...,2018.0,NaN,12,12,12.0,NaN,0,0,0.0,NaN
3,68,1,Route_160,low_frequency,12583,-8.031478,-34.958622,3171.0,12021,35.0,...,2018.0,NaN,12,12,12.0,NaN,0,0,0.0,NaN
4,68,1,Route_160,low_frequency,12589,-8.036750,-34.946086,4838.0,12021,43.0,...,2018.0,NaN,12,12,12.0,NaN,0,0,0.0,NaN


In [10]:
# TotalInput = input_file[:1000]
# TotalInput.shape

(1000, 60)

In [11]:
# del input_file

In [13]:
# FinalLabel = TotalInput['busBunching']
# TotalInput.drop('busBunching', axis=1, inplace=True)

In [7]:
# NoClass = FinalLabel.nunique()
# Threshold = 50

NoClass = len(list(set(np.ndarray.flatten(FinalLabel))))
Threshold = len(TotalInput[0, 0, :, 0])

# Making training and test data: 80% Training, 20% Test
Train_X, Test_X, Train_Y, Test_Y_ori = train_test_split(TotalInput, FinalLabel, test_size=0.20, random_state=7)

Train_Y = keras.utils.to_categorical(Train_Y, num_classes=NoClass)
Test_Y = keras.utils.to_categorical(Test_Y_ori, num_classes=NoClass)

In [8]:
# Model and Compile
model = Sequential()
activ = 'relu'
model.add(Conv2D(10, (1, 5), strides=(1, 1), padding='same', activation=activ, input_shape=(1, Threshold, 5)))
model.add(Conv2D(10, (1, 5), strides=(1, 1), padding='same', activation=activ))
model.add(MaxPooling2D(pool_size=(1, 2)))

model.add(Conv2D(22, (1, 5), strides=(1, 1), padding='same', activation=activ))
model.add(Conv2D(22, (1, 5), strides=(1, 1), padding='same', activation=activ))
model.add(MaxPooling2D(pool_size=(1, 2)))

model.add(Conv2D(41, (1, 5), strides=(1, 1), padding='same', activation=activ))
model.add(Conv2D(41, (1, 5), strides=(1, 1), padding='same', activation=activ))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Dropout(.5))

model.add(Flatten())
A = model.output_shape
model.add(Dense(int(A[1] * 1/5.), activation=activ))
model.add(Dropout(.5))

model.add(Dense(NoClass, activation='softmax'))

W0718 12:55:02.322429 140508683405120 deprecation_wrapper.py:119] From /home/veruska/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0718 12:55:02.323993 140508683405120 deprecation_wrapper.py:119] From /home/veruska/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0718 12:55:02.561687 140508683405120 deprecation_wrapper.py:119] From /home/veruska/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0718 12:55:02.651026 140508683405120 deprecation_wrapper.py:119] From /home/veruska/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0718 12:55:02.

In [9]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

offline_history = model.fit(Train_X, Train_Y, epochs=10, batch_size=64, shuffle=False,
                            validation_data=(Test_X, Test_Y))
hist = offline_history

W0718 12:55:05.027034 140508683405120 deprecation_wrapper.py:119] From /home/veruska/.local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0718 12:55:05.031805 140508683405120 deprecation_wrapper.py:119] From /home/veruska/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0718 12:55:05.247287 140508683405120 deprecation.py:323] From /home/veruska/.local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 59s 1ms/step - loss: 1.3835 - acc: 0.9141 - val_loss: 1.2685 - val_acc: 0.9213
Epoch 2/10
40000/40000 [==============================] - 58s 1ms/step - loss: 1.3338 - acc: 0.9172 - val_loss: 1.2685 - val_acc: 0.9213
Epoch 3/10
40000/40000 [==============================] - 58s 1ms/step - loss: 1.3338 - acc: 0.9172 - val_loss: 1.2685 - val_acc: 0.9213
Epoch 4/10
40000/40000 [==============================] - 58s 1ms/step - loss: 1.3338 - acc: 0.9172 - val_loss: 1.2685 - val_acc: 0.9213
Epoch 5/10
40000/40000 [==============================] - 58s 1ms/step - loss: 1.3338 - acc: 0.9172 - val_loss: 1.2685 - val_acc: 0.9213
Epoch 6/10
40000/40000 [==============================] - 58s 1ms/step - loss: 1.3338 - acc: 0.9172 - val_loss: 1.2685 - val_acc: 0.9213
Epoch 7/10
40000/40000 [==============================] - 58s 1ms/step - loss: 1.3338 - acc: 0.9172 - val_loss: 1.2685 - val_acc

In [10]:
# Saving the test and training score for varying number of epochs.
# with open('Revised_accuracy_history_largeEpoch_NoSmoothing.pickle', 'wb') as f:
#     pickle.dump([hist.epoch, hist.history['acc'], hist.history['val_acc']], f)

A = np.argmax(hist.history['val_acc'])
print('the optimal epoch size: {}, the value of high accuracy {}'.format(hist.epoch[A], np.max(hist.history['val_acc'])))

# Calculating the test accuracy, precision, recall
Pred = model.predict(Test_X, batch_size=32)
Pred_Label = np.argmax(Pred, axis=1)

the optimal epoch size: 0, the value of high accuracy 0.9213


In [11]:
1 in Pred_Label # the model isn't predicting bus bunching

False

In [31]:
ActualPositive

[array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  92,
         93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
        106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
        119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
        132, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145,
        146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158,
        159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
        172, 173, 174, 175, 176, 178, 179, 180, 181

In [32]:
PredictedPositive

[array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [54]:
TruePositive

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  14,
  15,
  16,
  17,
  19,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  36,
  37,
  38,
  40,
  41,
  42,
  43,
  44,
  45,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  67,
  68,
  69,
  70,
  72,
  73,
  74,
  75,
  76,
  78,
  79,
  80,
  82,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  100,
  101,
  102,
  103,
  105,
  106,
  107,
  108,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  129,
  130,
  131,
  132,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  145,
  146,
  147,
  148,
  149,
  150,
  154,
  155,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  167,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  178,
  179,
  180,

In [55]:
FalsePositive

[[13,
  18,
  20,
  35,
  39,
  46,
  66,
  71,
  77,
  81,
  83,
  99,
  104,
  109,
  128,
  133,
  143,
  144,
  151,
  152,
  153,
  156,
  166,
  177,
  197,
  200,
  205,
  215,
  222,
  224,
  228,
  232,
  239,
  240,
  243,
  256,
  268,
  292,
  294,
  300,
  301,
  305,
  307,
  308,
  315,
  334,
  338,
  340,
  362,
  366,
  372,
  387,
  439,
  454,
  483,
  494,
  499,
  513,
  520,
  524,
  527,
  531,
  567,
  576,
  578,
  593,
  601,
  608,
  612,
  625,
  626,
  629,
  636,
  637,
  639,
  662,
  696,
  715,
  723,
  724,
  730,
  735,
  742,
  745,
  753,
  761,
  777,
  785,
  817,
  828,
  830,
  842,
  864,
  872,
  875,
  876,
  883,
  884,
  897,
  905,
  908,
  919,
  924,
  940,
  942,
  958,
  976,
  998,
  1007,
  1023,
  1032,
  1038,
  1042,
  1056,
  1062,
  1065,
  1088,
  1099,
  1104,
  1105,
  1109,
  1111,
  1116,
  1127,
  1133,
  1148,
  1152,
  1153,
  1156,
  1168,
  1185,
  1217,
  1240,
  1246,
  1252,
  1253,
  1268,
  1274,
  1277,
  1283,


In [12]:
ActualPositive = []
for i in range(NoClass):
    AA = np.where(Test_Y_ori == i)[0]
    ActualPositive.append(AA)

PredictedPositive = []
for i in range(NoClass):
    AA = np.where(Pred_Label == i)[0]
    PredictedPositive.append(AA)

TruePositive = []
FalsePositive = []
for i in range(NoClass):
    AA = []
    BB = []
    for j in PredictedPositive[i]:
        if Pred_Label[j] == Test_Y_ori[j]:
            AA.append(j)
        else:
            BB.append(j)
    TruePositive.append(AA)
    FalsePositive.append(BB)
Precision = []
Recall = []


precision = ( len(TruePositive[0]) + len(TruePositive[1]) ) / ( len(PredictedPositive[0]) + len(PredictedPositive[1]) )
recall = ( len(TruePositive[0]) + len(TruePositive[1]) ) / ( len(ActualPositive[0]) + len(ActualPositive[1]) )
                                                            
# for i in range(NoClass):
#     Precision.append(len(TruePositive[i]) * 1./len(PredictedPositive[i]))
#     Recall.append(len(TruePositive[i]) * 1./len(ActualPositive[i]))

ConfusionM = confusion_matrix(list(Test_Y_ori), Pred_Label, labels=[0, 1, 2, 3, 4])

In [13]:
print('Confusion Matrix: ', ConfusionM)
print("Recall", recall)
print('precision', precision)
print(time.clock() - start_time, "seconds")

Confusion Matrix:  [[9213    0    0    0    0]
 [ 787    0    0    0    0]
 [   0    0    0    0    0]
 [   0    0    0    0    0]
 [   0    0    0    0    0]]
Recall 0.9213
precision 0.9213
4251.375491 seconds
